# Creating DataFrame
<br> 1. Loading RAW data
<br> 2. Converting it to DataFrame and creating relevant columns from the content
<br> 3. Deleting rows with empty values
<br> 4. Save the DataFrame to a file for easy opening (no need to convert it every time)


In [1]:
import email
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output

In [2]:
pd.options.mode.chained_assignment = None
emails_df = pd.read_csv('data/emails.csv')


In [3]:
emails_df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [4]:
def get_text_from_email(msg: email.message.Message) -> str:
    """To get the content from email objects
    :param msg: the email object
    :return: the content of the email
    """
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line: str) -> frozenset:
    """
    To separate multiple email addresses
    :param line: the line of the csv file
    :return: a set of email addresses
    """
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [5]:

# Parse the emails into a list email objects
messages = list(map(email.message_from_string, emails_df['message']))
emails_df.drop('message', axis=1, inplace=True)

# Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    emails_df[key] = [doc[key] for doc in messages]

# Parse content from emails
emails_df['content'] = list(map(get_text_from_email, messages))

# Split multiple email addresses
emails_df['From'] = emails_df['From'].map(split_email_addresses)
emails_df['To'] = emails_df['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
emails_df['user'] = emails_df['file'].map(lambda x:x.split('/')[0])
del messages

In [6]:
emails_df.isnull().sum()

file                             0
Message-ID                       0
Date                             0
From                             0
To                           21847
Subject                          0
Mime-Version                    29
Content-Type                    29
Content-Transfer-Encoding       29
X-From                          29
X-To                            29
X-cc                            29
X-bcc                           29
X-Folder                        29
X-Origin                        29
X-FileName                      29
content                          0
user                             0
dtype: int64

In [7]:
#emails_df find null rows
emails_df.loc[emails_df['To'].isnull()]

,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
188,allen-p/_sent_mail/264.,<15201149.1075855691021.JavaMail.evans@thyme>,"Mon, 1 May 2000 03:56:00 -0700 (PDT)",(phillip.allen@enron.com),None,Re: DSL- Installs,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Circuit Provisioning@ENRON,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,No one will be home on 5/11/00 to meet DSL ins...,allen-p
603,allen-p/all_documents/10.,<21975671.1075855665520.JavaMail.evans@thyme>,"Wed, 13 Dec 2000 08:35:00 -0800 (PST)",(messenger@ecm.bloomberg.com),None,Bloomberg Power Lines Report,1.0,text/plain; charset=ANSI_X3.4-1968,quoted-printable,"""Bloomberg.com"" <messenger@ecm.bloomberg.com>",(undisclosed-recipients),,,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,Here is today's copy of Bloomberg Power Lines....,allen-p
781,allen-p/all_documents/263.,<9828978.1075855671241.JavaMail.evans@thyme>,"Mon, 1 May 2000 03:56:00 -0700 (PDT)",(phillip.allen@enron.com),None,Re: DSL- Installs,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Circuit Provisioning@ENRON,,,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,No one will be home on 5/11/00 to meet DSL ins...,allen-p
873,allen-p/all_documents/348.,<8236042.1075855673105.JavaMail.evans@thyme>,"Fri, 7 Jan 2000 16:23:00 -0800 (PST)",(owner-strawbale@crest.org),None,,1.0,text/plain; charset=us-ascii,7bit,owner-strawbale@crest.org,"undisclosed-recipients:,",,,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,<4DDE116DBCA1D3118B130080C840BAAD02CD53@ppims....,allen-p
885,allen-p/all_documents/359.,<26959382.1075855693279.JavaMail.evans@thyme>,"Mon, 14 May 2001 09:04:00 -0700 (PDT)",(messenger@ecm.bloomberg.com),None,Bloomberg Power Lines Report,1.0,text/plain; charset=ANSI_X3.4-1968,quoted-printable,"""Bloomberg.com"" <messenger@ecm.bloomberg.com>",(undisclosed-recipients),,,\Phillip_Allen_June2001\Notes Folders\All docu...,Allen-P,pallen.nsf,Here is today's copy of Bloomberg Power Lines....,allen-p
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516846,zufferli-j/calendar/1.,<28390332.1075842024763.JavaMail.evans@thyme>,"Tue, 5 Feb 2002 09:43:01 -0800 (PST)",(john.zufferli@enron.com),None,,1.0,text/plain; charset=us-ascii,7bit,"Zufferli, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...",,,,"\ExMerge - Zufferli, John\Calendar",ZUFFERLI-J,john zufferli 6-26-02.PST,Conference call with UBS,zufferli-j
516867,zufferli-j/deleted_items/112.,<32471560.1075842024421.JavaMail.evans@thyme>,"Fri, 25 Jan 2002 14:18:55 -0800 (PST)",(no.address@enron.com),None,Weekend Outage Report for 1/25/02-1/27/02,1.0,text/plain; charset=us-ascii,7bit,Enron Change Management Announcement@ENRON,Houston Outage Report@ENRON,,,"\ExMerge - Zufferli, John\Deleted Items",ZUFFERLI-J,john zufferli 6-26-02.PST,______________________________________________...,zufferli-j
516878,zufferli-j/deleted_items/123.,<24228255.1075842024716.JavaMail.evans@thyme>,"Mon, 21 Jan 2002 16:59:41 -0800 (PST)",(david_paul_smith@attbi.com),None,"""less busy"" Head Trip flyer",1.0,text/plain; charset=us-ascii,7bit,david_paul_smith@attbi.com@ENRON,undisclosed-recipients:;@ENRON,,,"\ExMerge - Zufferli, John\Deleted Items",ZUFFERLI-J,john zufferli 6-26-02.PST,"Howdy!\n\nHere's a better ""less busy"" flyer to...",zufferli-j
516927,zufferli-j/deleted_items/58.,<27830378.1075842022474.JavaMail.evans@thyme>,"Thu, 31 Jan 2002 13:22:01 -0800 (PST)",(no.address@enron.com),None,Weekend Outage Report for 2/1/02 - 2/3/02,1.0,text/plain; charset=us-ascii,7bit,Enron Change Management Announcement@ENRON,Houston Outage Report@ENRON,,,"\ExMerge - Zufferli, John\Deleted Items",ZUFFERLI-J,john zufferli 6-26-02.PST,______________________________________________...,zufferli-j


In [8]:
# remove rows with null values
emails_df = emails_df.dropna()

In [9]:
emails_df.isnull().sum()


file                         0
Message-ID                   0
Date                         0
From                         0
To                           0
Subject                      0
Mime-Version                 0
Content-Type                 0
Content-Transfer-Encoding    0
X-From                       0
X-To                         0
X-cc                         0
X-bcc                        0
X-Folder                     0
X-Origin                     0
X-FileName                   0
content                      0
user                         0
dtype: int64

In [10]:
emails_df.shape

(495547, 18)

In [ ]:
# save dataframe to file
emails_df.to_csv('data/emails_df.csv', index=False)

In [ ]:
# open dataframe from file
emails_df_2 = pd.read_csv('data/emails_df.csv')

In [ ]:
emails_df_2.head()

,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
0,allen-p/_sent_mail/1.,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",frozenset({'phillip.allen@enron.com'}),frozenset({'tim.belden@enron.com'}),NaN,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast\n\n,allen-p
1,allen-p/_sent_mail/10.,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",frozenset({'phillip.allen@enron.com'}),frozenset({'john.lavorato@enron.com'}),Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p
2,allen-p/_sent_mail/100.,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",frozenset({'phillip.allen@enron.com'}),frozenset({'leah.arsdall@enron.com'}),Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p
3,allen-p/_sent_mail/1000.,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",frozenset({'phillip.allen@enron.com'}),frozenset({'randall.gay@enron.com'}),NaN,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy,\n\n Can you send me a schedule of the s...",allen-p
4,allen-p/_sent_mail/1001.,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",frozenset({'phillip.allen@enron.com'}),frozenset({'greg.piper@enron.com'}),Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p
